# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234359399705                   -0.50    7.0         
  2   -7.249835193985       -1.81       -1.40    1.0   7.01ms
  3   -7.251104508452       -2.90       -1.99    2.0   7.84ms
  4   -7.251071839724   +   -4.49       -1.94    2.0   10.9ms
  5   -7.251326640236       -3.59       -2.56    1.0   6.98ms
  6   -7.251336743195       -5.00       -2.92    2.0   8.47ms
  7   -7.251338571987       -5.74       -3.43    1.0   7.05ms
  8   -7.251338767601       -6.71       -3.86    1.0   7.14ms
  9   -7.251338796721       -7.54       -4.43    2.0   8.90ms
 10   -7.251338798530       -8.74       -4.90    2.0   8.60ms
 11   -7.251338798685       -9.81       -5.43    2.0   8.77ms
 12   -7.251338798698      -10.88       -5.66    2.0   8.83ms
 13   -7.251338798704      -11.26       -6.22    1.0   12.0ms
 14   -7.251338798704      -12.23       -6.56    3.0   10.7ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03955420500263514
[ Info: Arnoldi iteration step 2: normres = 0.7350314144418543
[ Info: Arnoldi iteration step 3: normres = 0.7831586244304041
[ Info: Arnoldi iteration step 4: normres = 0.30083318878819426
[ Info: Arnoldi iteration step 5: normres = 0.3181963169215268
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.88e-02, 5.84e-02, 2.33e-01, 2.04e-01, 3.94e-02)
[ Info: Arnoldi iteration step 6: normres = 0.45647533738788865
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.32e-03, 1.66e-01, 4.07e-01, 1.02e-01, 6.13e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08313312669698986
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.37e-04, 2.87e-02, 1.39e-02, 3.47e-02, 5.97e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11077071926525388
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.62e